# ***Forecasting on Historical Sales Data***

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

import numpy as np

#!pip install tensorflow

import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv1D, MaxPooling1D, Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.layers import LSTM,Dense,Dropout,Input


In [2]:
df_retail_sales_train = pd.read_csv('/content/drive/MyDrive/DeepForecast/DataCollected/Retail_Store_Inventory.csv')

dataFrameForViz = pd.DataFrame()


def describeCollectedData(df):
    print(f'DataFrame Shape : ',df.shape)
    print(f'DataFrame Info')
    print(df.info())
    df.dropna()
    df.dropna(axis=1)
    print(f'DataFrame Shape after droping null values : ',df.shape)
    return df

data_df = describeCollectedData(df_retail_sales_train)
def preprocessingStoreData(df):
    df = df.drop(['Units Ordered','Inventory Level','Holiday/Promotion','Competitor Pricing'], axis=1)
    df.rename(columns={'Units Sold':'Sales','Product ID':'Product',
                       'Store ID':'Store',
                       'Demand Forecast':'Demand',
                       "Weather Condition":'Weather_Condition'},inplace=True)
    return df

preprocessedRetailStoreDf = preprocessingStoreData(data_df)

DataFrame Shape :  (73100, 15)
DataFrame Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                73100 non-null  object 
 1   Store ID            73100 non-null  object 
 2   Product ID          73100 non-null  object 
 3   Category            73100 non-null  object 
 4   Region              73100 non-null  object 
 5   Inventory Level     73100 non-null  int64  
 6   Units Sold          73100 non-null  int64  
 7   Units Ordered       73100 non-null  int64  
 8   Demand Forecast     73100 non-null  float64
 9   Price               73100 non-null  float64
 10  Discount            73100 non-null  int64  
 11  Weather Condition   73100 non-null  object 
 12  Holiday/Promotion   73100 non-null  int64  
 13  Competitor Pricing  73100 non-null  float64
 14  Seasonality         73100 non-null  object 
dtypes: floa

In [14]:

def prepareDataWithSpecificCategory(df, category):
    return df[(df['Category'] == category)]

def prepareDataWithSpecificProduct(df, product):
    return df[(df['Product'] == product)]

def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names

    # Drop rows with NaN values
    if dropnan:
        #agg.dropna(inplace=True)
        #print(agg.isnull().sum())
        agg.fillna(agg.mean(),inplace=True)

    return agg

def prepareForecast(preprocessedRetailStoreDf, category):
    df_poductCategory = prepareDataWithSpecificCategory(preprocessedRetailStoreDf,'Groceries')
    df_to_predict = df_poductCategory.sort_values('Date').groupby(['Date'], as_index=False)
    df_to_predict = df_to_predict.agg({'Sales':['mean']})
    df_to_predict.columns = ['Date','Sales']

    df_train = df_to_predict

    window = 179
    future_span = 30
    series = series_to_supervised(df_train.drop('Date', axis=1), window=window, lag=future_span)

    # Label
    labels_col = 'Sales(t+%d)' % future_span
    labels = series[labels_col]
    series = series.drop(labels_col, axis=1)

    scaler = StandardScaler()
    series = scaler.fit_transform(series)

    X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.4, random_state=0)
    X_train_series = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_valid_series = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], 1))
    print('Train set shape', X_train_series.shape)
    print('Validation set shape', X_valid_series.shape)
    dataFrameForViz['dayOfYear']=range(0,X_train_series.shape[0])

    executeModel(X_train_series, X_valid_series, Y_train, Y_valid,category)
    return dataFrameForViz


# ***CNN for Demand Forecasting***

In [6]:

def executeModel(X_train_series, X_valid_series, Y_train, Y_valid,Category):
  lr = 0.0003
  adam = tf.keras.optimizers.Adam(lr)

  model_cnn = keras.Sequential()
  model_cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
  model_cnn.add(MaxPooling1D(pool_size=8))
  model_cnn.add(Flatten())
  model_cnn.add(Dense(50, activation='relu'))
  model_cnn.add(Dense(50, activation='relu'))
  model_cnn.add(Dropout(0.2))
  model_cnn.add(Dense(32, activation='relu'))
  model_cnn.add(Dropout(0.2))
  model_cnn.add(Dense(1))
  model_cnn.compile(loss='mse', optimizer=adam)
  model_cnn.summary()

  epochs = 700
  batch = 10
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50,
        verbose=1, mode='auto', restore_best_weights=True)
  cnn_history = model_cnn.fit(X_train_series, Y_train,
    validation_data=(X_valid_series, Y_valid), epochs=epochs, verbose=2)

  train_pred = model_cnn.predict(X_train_series)
  valid_pred = model_cnn.predict(X_valid_series)

  dataFrameForViz[Category] = train_pred.flatten() # Flatten the train_pred array before assigning


In [7]:
for cat in preprocessedRetailStoreDf['Category'].unique():
  prepareForecast(preprocessedRetailStoreDf,cat)

#Exporting the Forecasted Data
dataFrameForViz.to_csv('/content/drive/MyDrive/DeepForecast/Output/RetailOnDemandForecast.csv', encoding='utf-8', index=False, header=True)


Train set shape (438, 180, 1)
Validation set shape (293, 180, 1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 179, 128)       │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 22, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2816)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 50)             │       140,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         1,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 145,449 (568.16 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_5 (Conv1D)               │ (None, 179, 128)       │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 22, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 2816)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 50)             │       140,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         1,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 145,449 (568.16 KB)

 Trainable params: 145,449 (568.16 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/700
14/14 - 2s - 168ms/step - loss: 19306.9473 - val_loss: 18492.7227
Epoch 2/700
14/14 - 0s - 32ms/step - loss: 18201.6797 - val_loss: 16883.2109
Epoch 3/700
14/14 - 0s - 21ms/step - loss: 16268.0215 - val_loss: 14198.2627
Epoch 4/700
14/14 - 0s - 22ms/step - loss: 13004.5400 - val_loss: 10106.5645
Epoch 5/700
14/14 - 0s - 28ms/step - loss: 8545.6289 - val_loss: 5371.2041
Epoch 6/700
14/14 - 0s - 20ms/step - loss: 4454.5142 - val_loss: 2285.2390
Epoch 7/700
14/14 - 0s - 20ms/step - loss: 2837.1323 - val_loss: 2008.3744
Epoch 8/700
14/14 - 0s - 21ms/step - loss: 2697.1938 - val_loss: 1931.5679
Epoch 9/700
14/14 - 0s - 33ms/step - loss: 2530.4885 - val_loss: 1868.9484
Epoch 10/700
14/14 - 1s - 40ms/step - loss: 2510.5237 - val_loss: 1833.7993
Epoch 11/700
14/14 - 1s - 46ms/step - loss: 2533.8267 - val_loss: 1801.3436
Epoch 12/700
14/14 - 1s - 47ms/step - loss: 2480.9070 - val_loss: 1761.4937
Epoch 13/700
14/14 - 1s - 45ms/step - loss: 2255.8040 - val_loss: 1723.5359
Epoch 14/700